In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from surrogate import ImageSurrogate

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

from datetime import datetime
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [5]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Load Data

In [6]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

### Load Data

In [7]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Resize
import cv2
x_train = np.array([cv2.resize(x,(224,224)) for x in x_train])
x_val = np.array([cv2.resize(x,(224,224)) for x in x_val])
x_test = np.array([cv2.resize(x,(224,224)) for x in x_test])

# Preprocess
x_train = tf.keras.applications.resnet50.preprocess_input(x_train)
x_val = tf.keras.applications.resnet50.preprocess_input(x_val)
x_test = tf.keras.applications.resnet50.preprocess_input(x_test) 

50000 train samples
5000 val samples
5000 test samples


## Load Model

In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20210514_14_39_21/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# CXPlain

In [17]:
sys.path.insert(1, '../../../cxplain')
from tensorflow.python.keras.losses import categorical_crossentropy
from cxplain.backend.masking.zero_masking import ZeroMasking
from cxplain.backend.tensorflow_cxplain import TensorflowCXPlain as CXPlain

In [18]:
downsample_factors = (16, 16)
batch_size = 32
learning_rate=0.001
masking_operation = ZeroMasking()
loss = categorical_crossentropy

In [19]:
## ResNet Explainer Model Builder Class
from cxplain.backend.model_builders.base_model_builder import BaseModelBuilder
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model


class ResNetModelBuilder(BaseModelBuilder):
    def __init__(self, downsample_factors, 
                 callbacks=list([]), early_stopping_patience=12,
                 batch_size=64, num_epochs=100, validation_fraction=0.1, shuffle=True,
                 learning_rate=0.0001, optimizer=None, verbose=0):
        super(ResNetModelBuilder, self).__init__(callbacks, early_stopping_patience, batch_size, num_epochs,
                                                 validation_fraction, shuffle, learning_rate, optimizer, verbose)
        self.downsample_factors = downsample_factors
        self.num_output_channels = 1

    def build(self, input_layer):
        downsampling_factor = int(np.prod(self.downsample_factors))
        last_layer = input_layer
        
        base_model = ResNet50(
            include_top=False, weights='imagenet', 
            input_shape=(224,224,3)
        )
        base_model = Model(base_model.input, base_model.get_layer('conv4_block3_2_conv').output)
        base_model.trainable = True

        last_layer = base_model(last_layer)

        # Learn Phi 
        last_layer = Conv2D(1, 1)(last_layer)
        
        return last_layer

In [20]:
model_input = Input((224,224,3))
base_model = ResNet50(
            include_top=False, weights='imagenet', 
            input_shape=(224,224,3)
        )
base_model = Model(base_model.input, base_model.get_layer('conv4_block3_2_conv').output)
base_model.trainable = True

last_layer = base_model(model_input)

# Learn Phi 
last_layer = Conv2D(1, 1)(last_layer)

m = Model(model_input, last_layer)

In [21]:
model_builder = ResNetModelBuilder(downsample_factors, verbose=1,
                                   batch_size=batch_size, learning_rate=learning_rate)

explainer = CXPlain(model, model_builder, masking_operation, loss, 
                    num_models=1, downsample_factors=downsample_factors, flatten_for_explained_model=False)

In [22]:
save_dir = 'cxplain'
model_dir = os.path.join(os.getcwd(), save_dir, 'results')
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

In [23]:
t = time.time()
explainer.fit(x_train, y_train)
training_time = time.time() - t

with open(os.path.join(model_dir, 'training_time.pkl'), 'wb') as f:
    pickle.dump(training_time, f)

  0%|          | 0/625 [00:00<?, ?it/s]

Created Masked Data
Fitting Explainer Model
Train on 18000 samples, validate on 2000 samples
Epoch 1/100
18000/18000 [==============================] - 179s 10ms/sample - loss: 1.7088 - lambda_6_loss: 1.6931 - all_loss: 0.0074 - lambda_7_loss: 0.0081 - val_loss: 1.6812 - val_lambda_6_loss: 1.6675 - val_all_loss: 0.0070 - val_lambda_7_loss: 0.0078
Epoch 2/100
18000/18000 [==============================] - 65s 4ms/sample - loss: 1.6339 - lambda_6_loss: 1.6188 - all_loss: 0.0074 - lambda_7_loss: 0.0081 - val_loss: 1.6313 - val_lambda_6_loss: 1.6175 - val_all_loss: 0.0070 - val_lambda_7_loss: 0.0077
Epoch 3/100
18000/18000 [==============================] - 65s 4ms/sample - loss: 1.5941 - lambda_6_loss: 1.5787 - all_loss: 0.0074 - lambda_7_loss: 0.0081 - val_loss: 1.5969 - val_lambda_6_loss: 1.5837 - val_all_loss: 0.0069 - val_lambda_7_loss: 0.0077
Epoch 4/100
18000/18000 [==============================] - 65s 4ms/sample - loss: 1.5579 - lambda_6_loss: 1.5426 - all_loss: 0.0074 - lambda_7_

# Explain w/ CXPlain

### Load Images

In [24]:
images_dir = os.path.join(os.getcwd(), 'images')
images = np.load(os.path.join(images_dir, 'processed_images.npy'), allow_pickle=True)

### Explain

In [27]:
t = time.time()
explanations = explainer.explain(images)
explaining_time = time.time() - t

### Save

In [28]:
with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)
    
explanations.dump(os.path.join(model_dir, 'explanations.npy'))